In [1]:
import pandas as pd
from datetime import datetime
from prettytable import PrettyTable

data = pd.read_csv("2008.csv")
carrier = pd.read_csv("AirlineCodes.csv")

current_month = datetime.now().month
current_day = datetime.now().day
current_time = datetime.now().time()

itinerary = data.loc[(data['Month'] == current_month) & (data['DayofMonth'] == current_day), ['UniqueCarrier', 'DepTime', 'ArrTime', 'FlightNum', 'Origin', 'Dest']]
itinerary = itinerary.dropna() 

carrier_mapping = dict(zip(carrier['Code'], carrier['Name']))

delays = data[['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']]
cancellations = data[['Cancelled']]

def parse_time(time_str):
    try:
        return datetime.strptime(str(time_str).split('.')[0], "%H%M").time()
    except ValueError:
        pass
    
def has_delay(row):
    for column in delays:
        if pd.notna(data.loc[row.name, column]) and data.loc[row.name, column] >= 15:
            return True
    return False

def cancelled(row):
    for column in cancellations:
        if pd.notna(data.loc[row.name, column]) and data.loc[row.name, column] != 0:
            return True
    return False

def get_airline(carrier_code):
    if carrier_code in carrier_mapping:
        return carrier_mapping[carrier_code]
    return "Void"


def get_status(row):
    if has_delay(row):
        return "Delayed"
    elif cancelled(row):
        return "Cancelled"
    else:
        return "On Time"

def format_time(time):
    return time.strftime("%I:%M %p")

def generate_arrival_board(rows):
    tableA = PrettyTable()
    tableA.field_names = ["Airline", "Flight", "Origin", "Status", "Time"]
    
    for _, row in rows.iterrows():
        arr_time = parse_time(row['ArrTime'])
        dep_time = parse_time(row['DepTime'])
        if arr_time is not None:
            if current_time >= dep_time and current_time <= arr_time: #arrivals
                tableA.add_row([
                    get_airline(row['UniqueCarrier']),
                    row['FlightNum'],
                    row['Origin'],
                    get_status(row),
                    format_time(arr_time) ])
    return tableA


def generate_departure_board(rows):
    tableD = PrettyTable()
    tableD.field_names = ["Airline", "Flight", "Destination", "Status", "Time"]

    for _, row in rows.iterrows():
        dep_time = parse_time(row['DepTime'])
        if dep_time is not None:
            if current_time <= dep_time: #departures
                tableD.add_row([
                    get_airline(row['UniqueCarrier']),
                    row['FlightNum'],
                    row['Dest'],
                    get_status(row),
                    format_time(dep_time) ])
    return tableD

def arrivals(airport):
    filtered_rows = itinerary[itinerary['Dest'] == airport]
    return generate_arrival_board(filtered_rows)

def departures(airport):
    filtered_rows = itinerary[itinerary['Origin'] == airport]
    return generate_departure_board(filtered_rows)

arrival_board = arrivals('MYR')
departure_board = departures('MYR')
display_time = format_time(current_time)

print(display_time)

print("\nArrivals:")
print(arrival_board)

print("\nDepartures:")
print(departure_board)


12:23 PM

Arrivals:
+------------+--------+--------+---------+----------+
|  Airline   | Flight | Origin |  Status |   Time   |
+------------+--------+--------+---------+----------+
|    PSA     |  5635  |  LGA   | On Time | 01:06 PM |
| US Airways |  1821  |  CLT   | On Time | 12:24 PM |
+------------+--------+--------+---------+----------+

Departures:
+--------------------+--------+-------------+---------+----------+
|      Airline       | Flight | Destination |  Status |   Time   |
+--------------------+--------+-------------+---------+----------+
|        Mesa        |  7125  |     IAD     | Delayed | 02:58 PM |
|        PSA         |  5636  |     LGA     | On Time | 01:25 PM |
|     US Airways     |  1065  |     CLT     | On Time | 07:47 PM |
|     US Airways     |  1109  |     CLT     | On Time | 12:50 PM |
| Atlantic Southeast |  4172  |     ATL     | On Time | 02:35 PM |
| Atlantic Southeast |  4358  |     ATL     | On Time | 03:40 PM |
| Atlantic Southeast |  4814  |     ATL 